In [13]:
%matplotlib inline

import sys
import os
import pathlib
import xarray as xr
import palettable
import glob
import matplotlib.pyplot as plt
import json

sys.path.append('/g/data/mn51/users/mtb563/toolbox/modules')
import utils

sys.path.append('/g/data/mn51/users/mtb563/repos_collab/plotting_maps')
from acs_plotting_maps import plot_acs_hazard, regions_dict, cmap_dict, tick_dict
xr.set_options(keep_attrs=True)

# User defined
indices = ['HWAtx','HWD','HWN','TNle02','TXge35','TXge40','TXge45','TXm','TXx','HWF']


In [14]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

def hazard_cmaps(key):
    cmaps = {
        'tasmax' : ListedColormap(colors=['#E3F4FB','#C8DEE8','#91C4EA','#56B6DC','#00A2AC','#30996C','#7FC69A','#B9DA88','#DCE799','#FCE850','#EACD44','#FED98E','#F89E64','#E67754','#D24241','#AD283B','#832D57','#A2667A','#AB9487'],name='tasmax'),
        'FFDI': palettable.colorbrewer.sequential.YlOrRd_9.mpl_colormap,
        'EHF': ListedColormap(colors=['#FEFEFE','#FEFEBE','#FEAC00','#EC5000','#EC5000'],name='EHF') 
    }
    return cmaps[key]


In [15]:
cmaps_ticks = {
    'HWN':   [ palettable.colorbrewer.sequential.YlOrRd_9.mpl_colormap, [1,2,3,4,5,6,7,8,9,10] ], 
    'HWF':   [ palettable.colorbrewer.sequential.YlOrRd_9.mpl_colormap, [15,30,45,60,75,90,105,120,135,150,165] ], 
    'HWAtx': [ hazard_cmaps('tasmax'), [18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52] ],
    'HWD':   [ palettable.colorbrewer.sequential.YlOrBr_9.mpl_colormap, [10,20,30,40,50,60,70,80,90,100] ],
    'TXge35':  [ palettable.colorbrewer.sequential.YlOrRd_9.mpl_colormap, [1,5,10,15,20,40,80,120,160,200] ],
    'TNle02':  [ palettable.colorbrewer.sequential.YlGnBu_9.mpl_colormap, [1,5,10,15,20,40,80,120,160,200] ]
    }

cmaps_ticks.update({'TXge40':cmaps_ticks['TXge35'], 'TXge45':cmaps_ticks['TXge35'], 'TXx':cmaps_ticks['HWAtx'],'TXm':cmaps_ticks['HWAtx']})


In [ ]:
for index in indices:
    for bc_method in ['bias-corrected','raw']:
        for aggr in ['individual_models','ensemble']:

            assert index in cmaps_ticks.keys(), f"Update cmaps_ticks to include {index}"
            
            ddir = f"/g/data/ia39/ncra/heat/data/{index}/{bc_method}/{aggr}/GWL-average/"
            fdir = f"/g/data/ia39/ncra/heat/figures/{index}/{bc_method}/{aggr}/GWL-average/"
            pathlib.Path(fdir).mkdir(parents=True, exist_ok=True)
            
            fnames = glob.glob(ddir+'*.nc')
            
            for f in fnames:
                print(f)
                ofile = fdir+f.rsplit('/', 1)[1].replace('.nc','.png')

                if not os.path.exists(ofile):
                    ds = utils.generalio.read_data(infiles=[f])
                    units = ds[index].units
                    if units == 'degC':
                        units = u'\N{DEGREE SIGN}C'
                    if index in ['HWF']:
                        units = '(days/year)'
                    fig,ax = plot_acs_hazard(data = ds[index],
                                             cmap = cmaps_ticks[index][0],
                                             ticks = cmaps_ticks[index][1],
                                             #tick_labels = cmaps_ticks[index][1],
                                             cbar_label = units,
                                             cbar_extend = "both",
                                             title = f"{ds[index].attrs['long_name'].capitalize()}",
                                             dataset_name = f"Models: ensemble, Bias-adjustment: {ds.bias_correction_method}" if aggr == 'ensemble' else f"GCM: {ds.driving_model}, RCM: {ds.downscaling_model}, Bias-adjustment: {ds.bias_correction_method}",
                                             date_range = ds.GWL.replace('GWL','Global Warming Level').replace('degC',u'\N{DEGREE SIGN}C'),
                                             watermark=None,
                                             outfile = 'temp.png',
                                                     );
                    metadata = ds.attrs
                    metadata['data-history'] = metadata.pop('history')
                    metadata['figure-history'] = utils.provenance.get_notebook_prov()
                    
                    plt.savefig(ofile,transparent=True,dpi=300,metadata={'provenance':json.dumps(metadata)})
                    plt.close()
                    del(fig,ax,ds,units,metadata,ofile)
